In [ ]:
import torch
import torch.nn as nn

In [ ]:
A = torch.rand(3,3)
print(A)

In [ ]:
a = torch.randn(1,3,3)
print(a.shape,a)
b = torch.randn(1,3,3)
print(b.shape,b)
c = [a,b]
print(c)
d = torch.cat(b)
print(b.shape,b)

In [ ]:
def ConvLayer(ks, inp_ch, out_ch):
    conv = nn.Conv2d(in_channels=inp_ch, out_channels=out_ch, kernel_size=ks, padding=1)
    return conv

class VGG(nn.Module):
    def __init__(self, mode='A'):
        super(VGG, self).__init__()
        self.mode = mode
        mode_dict = {"A":1, "B":2, "C":3, "D":4, "E":5}
        self.mode_dict = mode_dict
        self.conv11 = ConvLayer(3, 3, 64)
        if mode_dict[mode] >= 2:
            self.conv12 = ConvLayer(3, 64, 64)
        self.max1 = nn.MaxPool2d(kernel_size=2, stride=2)
        ################################
        self.conv21 = ConvLayer(3, 64, 128)
        if mode_dict[mode] >= 2:
            self.conv22 = ConvLayer(3, 128, 128)
        self.max2 = nn.MaxPool2d(kernel_size=2, stride=2)
        ################################
        self.conv31 = ConvLayer(3, 128, 256)
        self.conv32 = ConvLayer(3, 256, 256)
        if mode_dict[mode] == 3:
            self.conv33 = ConvLayer(1, 256, 256)
        if mode_dict[mode] >= 4:
            self.conv33 = ConvLayer(3, 256, 256)
        if mode_dict[mode] == 5:
            self.conv34 = ConvLayer(3, 256, 256)
        self.max3 = nn.MaxPool2d(kernel_size=2, stride=2)
        ################################
        self.conv41 = ConvLayer(3, 256, 512)
        self.conv42 = ConvLayer(3, 512, 512)
        if mode_dict[mode] == 3:
            self.conv43 = ConvLayer(1, 512, 512)
        if mode_dict[mode] >= 4:
            self.conv43 = ConvLayer(3, 512, 512)
        if mode_dict[mode] == 5:
            self.conv44 = ConvLayer(3, 512, 512)
        self.max4 = nn.MaxPool2d(kernel_size=2, stride=2)
        ################################
        self.conv51 = ConvLayer(3, 512, 512)
        self.conv52 = ConvLayer(3, 512, 512)
        if mode_dict[mode] == 3:
            self.conv53 = ConvLayer(1, 512, 512)
        if mode_dict[mode] >= 4:
            self.conv53 = ConvLayer(3, 512, 512)
        if mode_dict[mode] == 5:
            self.conv54 = ConvLayer(3, 512, 512)
        self.max5 = nn.MaxPool2d(kernel_size=2, stride=2)
        ################################
        self.fc1 = nn.Linear(25088, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 1000)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.conv11(x)
        x = self.act(x)
        if self.mode_dict[self.mode] >= 2:
            x = self.conv12(x)
            x = self.act(x)
        x = self.max1(x)
        print(x.size())
        ################################
        x = self.conv21(x)
        x = self.act(x)
        if self.mode_dict[self.mode] >= 2:
            x = self.conv22(x)
            x = self.act(x)
        x = self.max2(x)
        print(x.size())
        ################################
        x = self.conv31(x)
        x = self.act(x)
        x = self.conv32(x)
        x = self.act(x)
        if self.mode_dict[self.mode] == 3:
            x = self.conv33(x)
            x = self.act(x)
        if self.mode_dict[self.mode] >= 4:
            x = self.conv33(x)
            x = self.act(x)
        if self.mode_dict[self.mode] == 5:
            x = self.conv34(x)
            x = self.act(x)
        x = self.max3(x)
        print(x.size())
        ################################
        x = self.conv41(x)
        x = self.act(x)
        x = self.conv42(x)
        x = self.act(x)
        if self.mode_dict[self.mode] == 3:
            x = self.conv43(x)
            x = self.act(x)
        if self.mode_dict[self.mode] >= 4:
            x = self.conv43(x)
            x = self.act(x)
        if self.mode_dict[self.mode] == 5:
            x = self.conv44(x)
            x = self.act(x)
        x = self.max4(x)
        print(x.size())
        ################################
        x = self.conv51(x)
        x = self.act(x)
        x = self.conv52(x)
        x = self.act(x)
        if self.mode_dict[self.mode] == 3:
            x = self.conv53(x)
            x = self.act(x)
        if self.mode_dict[self.mode] >= 4:
            x = self.conv53(x)
            x = self.act(x)
        if self.mode_dict[self.mode] == 5:
            x = self.conv54(x)
            x = self.act(x)
        x = self.max5(x)
        print(x.size())
        ################################
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.act(x)
        x = self.fc3(x)
        x = self.act(x)
        return nn.Softmax()(x)

In [ ]:
model = VGG()
x = torch.rand(1, 3, 224, 224)
out = model(x)

In [ ]:
def ConvLayer(ks:int, inp_ch:int, out_ch:int) -> nn.Module:
    conv = nn.Conv2d(inp_ch, out_ch, ks, padding='same')
    return conv

class BasicBlock(nn.Module):
    def __init__(self, ks, inp_ch, out_ch) -> None:
        super().__init__()
        self.conv1 = ConvLayer(ks, inp_ch, out_ch)
        self.conv2 = ConvLayer(ks, out_ch, out_ch)
        self.act = nn.ReLU()
    def forward(self, inp):
        x = self.conv1(inp)
        x = self.act(x)
        x = self.conv2(x)
        x = self.act(x)
        return x + inp

class ResidualBlock(nn.Module):
    def __init__(self, ks1, ks2, ch1, ch2) -> None:
        super().__init__()
        self.conv1 = ConvLayer(ks1, ch1, ch1)
        self.conv2 = ConvLayer(ks2, ch1, ch1)
        self.conv3 = ConvLayer(ks1, ch1, ch2)
        self.act = nn.ReLU()
    def forward(self, inp):
        x = self.conv1(inp)
        x = self.act(x)
        x = self.conv2(x)
        x = self.act(x)
        x = self.conv3(x)
        x = self.act(x)
        return x + inp

class ResNet(nn.Module):
    def __init__(self, block_list, block) -> None:
        super().__init__()
        # block_list example: [3, 4, 6, 3] for ResNet-34
        self.block_list = block_list
        total_block = []
        
        for i in range(self.block_list[0]):
            if block == BasicBlock:
                total_block.append(block(3, 64, 64))
            else:
                total_block.append(block(1, 3, 64, 256))
                
        for i in range(self.block_list[1]):
            if block == BasicBlock:
                total_block.append(block(3, 64, 128))
            else:
                pass
            
        for i in range(self.block_list[2]):
            if block == BasicBlock:
                total_block.append(block(3, 128, 256))

        for i in range(self.block_list[3]):
            if block == BasicBlock:
                total_block.append(block(3, 256, 512))
        
        self.conv_list = nn.Sequential(
            *total_block,
        )

    def forward(self, x):
        out = self.conv_list(x)
        return nn.Softmax()(out)

In [ ]:
def return_resnet(layer):
    if layer == 18:
        return ResNet([2,2,2,2], BasicBlock)
    elif layer == 34:
        return ResNet([3,4,6,3], BasicBlock)
    elif layer == 50:
        return ResNet([3,4,6,3], ResidualBlock)
    elif layer == 101:
        return ResNet([3,4,23,3], ResidualBlock)
    elif layer == 152:
        return ResNet([3,8,36,3], ResidualBlock)